In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


1. ## step 1.)라이브러리 및 데이터 불러오기 
- 라이브러리 버젼 확인

In [2]:
#from IPython.display import clear_output
#!pip#tall -U lazypredict
#!pip3 install -U pandas #Upgrading pandas


In [3]:
BASE_DIR = "/kaggle/input/spaceship-titanic/"
train = pd.read_csv(BASE_DIR + "train.csv")
test = pd.read_csv(BASE_DIR + "test.csv")
sample_submission = pd.read_csv(BASE_DIR + "sample_submission.csv")
train.shape, test.shape, sample_submission.shape

((8693, 14), (4277, 13), (4277, 2))

# 변수분석
- passengerId-> 승객번호, gggg 는 그룹으로 이주를 간 사람들을 뜻 하고, pp 는 그 그룹안에서의 숫자이다. 종종 그들은 가족이지만 항상 그런것은 아니다. 
- HomePlant- 

## step 2. 탐색적 자료 분석(EDA)
- 데이터 시각화
- 산점도,막대그래프 
- 그래프에 대해 설명을 달아야 함. 
- 약간의 데이터 전처리

In [4]:
train.head()



,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# step 3. 데이터 전처리
- Feature Engineering 
- ML 모형을 돌리기 위해 표준화 등 / 원핫-인코딩 
- 파생변수(도출변수)
 + 왜 이 변수를 만들었는지에 대한 우리의 설명 필요 .
- 교차검증
- 하이퍼파라미터 튜닝 
 

# step4. 머신러닝 모형 개발 
- 모형에 대한 설명 필요.
- 모형을 1-2개 사용 

# step 6. 제출 
- 제출 양식은 샘플 만들어드림


# 참고 
- 다른 사람의 code 설명을 쭉 따라침.
- 노트북 표절 방지 위해, 여러분들이 참조한 코드는 반드시 링크 걸어둘 것.
- 저자이름, 글 제목 , 링크주소

# 마감일 
- 4월 12일 17시 40분
- 제출 형태
 + Leaderborad 래잉 사진 캡쳐
 + 고용노동부 보고 양식